<a href="https://colab.research.google.com/github/natalie-ayers/Iraq-post-conflict-rebel-governance/blob/main/Iraq_drought.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rasterio
!pip install rasterstats
!pip install regionmask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.9 MB/s eta 0:00:00


In [16]:
import pandas as pd
import geopandas as gpd
import ee
import geemap
import ast
from shapely import geometry
from urllib.request import urlretrieve
import xarray as xr
import rasterio as rio
import rasterstats as rstats
import regionmask
import matplotlib.pyplot as plt


In [3]:
iraq_shp_adm0_loc = 'irq_admbnda_adm0_cso_itos_20190603.shp'
iraq_shp_adm0 = gpd.read_file(iraq_shp_adm0_loc)
iraq_shp_adm0.head(2)

,Shape_Leng,Shape_Area,ADM0_EN,ADM0_AR,ADM0_PCODE,ADM0_REF,ADM0ALT1EN,ADM0ALT2EN,ADM0ALT1AR,ADM0ALT2AR,date,validOn,validTo,geometry
0,35.691174,42.141484,Iraq,العراق,IQ,None,None,None,None,None,2019-05-30,2019-06-03,None,"POLYGON ((42.84654 37.34800, 42.85154 37.34657..."


In [4]:
iraq_shp_adm3_loc = 'irq_admbnda_adm3_cso_20190603.shp'
iraq_shp_adm3 = gpd.read_file(iraq_shp_adm3_loc)
iraq_adm3_filt = iraq_shp_adm3.loc[:,("Shape_Area",'ADM3_EN','ADM3_PCODE',
                                    'ADM2_EN','ADM2_PCODE','ADM1_EN','ADM1_PCODE',
                                    'geometry')]

print(iraq_adm3_filt.shape)
print(iraq_adm3_filt.crs)
iraq_adm3_filt.head(2)

(294, 8)
EPSG:4326


,Shape_Area,ADM3_EN,ADM3_PCODE,ADM2_EN,ADM2_PCODE,ADM1_EN,ADM1_PCODE,geometry
0,0.015754,Abi Gharaq,IQG07Q02N02,Al-Hilla,IQG07Q02,Babil,IQG07,"POLYGON ((44.36654 32.56190, 44.36466 32.55802..."
1,0.145883,Abu Dalf,IQG16Q01N02,Al-Daur,IQG16Q01,Salah Al-Din,IQG16,"POLYGON ((44.19124 34.77808, 44.21397 34.75600..."


## Obtain Drought Indices

In [73]:
def download_drought_nc(index_type, index_period):
  """
  Inputs:
    index_type: string, type of index to download, eg 'CHIRPS_GLEAM'
    index_period: months of SEPI to include, as 2 digit string, eg "03" for 3 months SPEI
  """
  output_file = f"{index_type}_spei{index_period}.nc"
  url = (
      "https://dap.ceda.ac.uk/badc/hydro-jules/data/Global_drought_indices/"
      f"{index_type}/spei{index_period}.nc"
  )
  urlretrieve(url, output_file)

  return output_file


def get_aoi(shp, world=True):
    """Takes a geopandas object and converts it to a lat/ lon
    extent

    Parameters
    -----------
    shp : GeoPandas GeoDataFrame
        A geodataframe containing the spatial boundary of interest
    world : boolean
        True if you want lat / long to represent sinusoidal (0-360 degrees)

    Returns
    -------
    Dictionary of lat and lon spatial bounds
    """

    lon_lat = {}
    # Get lat min, max
    aoi_lat = [float(shp.total_bounds[1]), float(shp.total_bounds[3])]
    aoi_lon = [float(shp.total_bounds[0]), float(shp.total_bounds[2])]

    if world:
        aoi_lon[0] = aoi_lon[0] + 360
        aoi_lon[1] = aoi_lon[1] + 360
    lon_lat["lon"] = aoi_lon
    lon_lat["lat"] = aoi_lat
    return lon_lat


def slice_data_time_loc(xr_da_full,start_date,end_date,geo_bounds,index_type):

  # Subset
  if index_type.startswith('CHIRPS'):
    spei_time_loc = spei_xr.sel(
        time=slice(start_date, end_date),
        lon=slice(adm3_bounds["lon"][0], adm3_bounds["lon"][1]),
        lat=slice(adm3_bounds["lat"][0], adm3_bounds["lat"][1])
        )
  # MSWEP indices in different format, need to swap lat min-max bounds
  elif index_type.startswith('MSWEP'):
    spei_time_loc = spei_xr.sel(
        time=slice(start_date, end_date),
        lon=slice(adm3_bounds["lon"][0], adm3_bounds["lon"][1]),
        lat=slice(adm3_bounds["lat"][1], adm3_bounds["lat"][0])
        )
  else:
    print("Index type not recognised")

  return spei_time_loc

In [59]:
def generate_spei_stats(xr_da_unmasked, region_mask, areas_gdf):
  xr_spei = xr_da_unmasked.where(region_mask)
  print('Created masked dataarray with dims',xr_spei.dims)

  # calculate mean spei
  mean_sum = xr_spei.groupby("time").mean(["lat", "lon"])
  mean_df = mean_sum.to_dataframe()
  mean_df.rename(columns = {'spei':'mean_spei'},inplace=True)

  # calculate max spei
  max_sum = xr_spei.groupby("time").max(["lat", "lon"])
  max_df = max_sum.to_dataframe()
  max_df.rename(columns = {'spei':'max_spei'},inplace=True)

  # calculate min spei
  min_sum = xr_spei.groupby("time").min(["lat", "lon"])
  min_df = min_sum.to_dataframe()
  min_df.rename(columns = {'spei':'min_spei'},inplace=True)

  # calculate points in drought
  perc_drought = xr_da_unmasked.\
                where(region_mask & (xr_da_unmasked < -1)).\
                groupby("time").count(["lat", "lon"])
  perc_drought_df = perc_drought.to_dataframe()
  perc_drought_df.rename(columns = {'spei':'num_drought'},inplace=True)

  # calculate total points in each area
  count_all = xr_spei.\
                  groupby("time").count(["lat", "lon"])
  count_all_df = count_all.to_dataframe()
  count_all_df.rename(columns = {'spei':'num_all'},inplace=True)

  # combine all dataframes
  summaries = pd.concat([mean_df, max_df, min_df,perc_drought_df,count_all_df], axis=1)
  summaries = summaries.reset_index()

  summaries = summaries.merge(areas_gdf.loc[:,('ADM3_EN','ADM3_PCODE')],how='outer',
                            left_on='region',right_on=areas_gdf.index)

  # confirm all regions assigned
  if summaries[summaries['ADM3_PCODE'].isna()].shape[0] > 0:
    print("df.shape with adm3_pcode assigned:",summaries[summaries['ADM3_PCODE'].isna()].shape)
    print("Should not be any areas unassigned; stopping and outputting summaries as-is to test")
    return summaries
  if summaries[summaries['ADM3_PCODE'].isna()].shape[0] > 0:
    print("df.shape where num drought points > num points in region:",summaries[summaries['ADM3_PCODE'].isna()].shape)
    print("Should not be any areas with more drought points than total points; stopping and outputting summaries as-is to test")

  print("Confirmed data matches tested expectations")

  try:
    summaries = summaries.drop(columns=['index'])
  except KeyError:
    print("No index column to drop")
    pass

  return summaries



In [74]:
def get_spei_df(index_type, index_period, areas_gdf,start_date,end_date):
  print(f"Downloading {index_type} SPEI {index_period} NetCDF from CEDA archive...")
  output_file = download_drought_nc(index_type, index_period)
  print(f"Downloaded and saved {output_file}")

  print(f"Opening NetCDF file to use")
  with xr.open_dataset(output_file) as ds:
    spei_xr = ds['spei']

  # using process from https://www.earthdatascience.org/courses/use-data-open-source-python/hierarchical-data-formats-hdf/subset-netcdf4-climate-data-spatially-aoi/
  print(f"Creating mask of areas of interest")
  adm3_mask = regionmask.mask_3D_geopandas(areas_gdf,
                                          spei_xr.lon,
                                          spei_xr.lat)

  print("Getting the bounds of areas of interest")
  adm3_bounds = get_aoi(areas_gdf, world=False)

  print("Limiting index data to desired dates and area boundaries")
  spei_time_loc = slice_data_time_loc(spei_xr,start_date,end_date,adm3_bounds,index_type)

  print("Generating summary statistics for SPEI values over desired areas")
  sums = generate_spei_stats(spei_time_loc, adm3_mask, areas_gdf)

  summary_spei_csv = f"{index_type}_{index_period}_spei_sums.csv"
  print(f"Saving summary statistics to file {summary_spei_csv}")
  sums.to_csv(summary_spei_csv,index=False)
  print(f"Done with {output_file} SPEI processing")
  print(" ")

In [ ]:
index_types = ['CHIRPS_GLEAM','CHIRPS_hPET','MSWEP_GLEAM','MSWEP_hPET']
index_periods = ['01','03','06']

for index_type in index_types:
  for index_period in index_periods:
    get_spei_df(index_type, index_period, iraq_adm3_filt,"2001-01-01",'2022-12-31')

Individual steps for testing:

In [64]:
output_file = download_drought_nc('MSWEP_GLEAM', '01')

In [65]:
with xr.open_dataset(output_file) as ds:
    spei_xr = ds['spei']

spei_xr

<xarray.DataArray 'spei' (time: 504, lat: 2800, lon: 7200)>
[10160640000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1981-01-01 1981-02-01 ... 2022-12-01
  * lon      (lon) float64 -180.0 -179.9 -179.9 -179.8 ... 179.9 179.9 180.0
  * lat      (lat) float64 85.0 84.95 84.9 84.85 ... -54.8 -54.85 -54.9 -54.95
Attributes:
    long_name:  Standardized Precipitation-Evapotranspiration Index
    units:      z-values

In [66]:
print(spei_xr['lat'].min(dim='lat'))
print(spei_xr['lat'].max(dim='lat'))

<xarray.DataArray 'lat' ()>
array(-54.95)
<xarray.DataArray 'lat' ()>
array(85.)


In [67]:
iraq_adm3_filt.geometry.total_bounds

array([38.7948362 , 29.06952694, 48.62379497, 37.37726401])

In [68]:
# using process from https://www.earthdatascience.org/courses/use-data-open-source-python/hierarchical-data-formats-hdf/subset-netcdf4-climate-data-spatially-aoi/

adm3_mask = regionmask.mask_3D_geopandas(iraq_adm3_filt,
                                         spei_xr.lon,
                                         spei_xr.lat)

In [5]:
# took 5.5 minutes to download spei01 from cirps_gleam
index_type = 'CHIRPS_hPET'
index_file = "spei01.nc"

output_file = f"{index_type}_{index_file}"
url = (
    "https://dap.ceda.ac.uk/badc/hydro-jules/data/Global_drought_indices/"
    f"{index_type}/{index_file}"
)
path, headers = urlretrieve(url, output_file)
for name, val in headers.items():
  print(name, val)

Date Fri, 15 Mar 2024 22:41:55 GMT
Content-Type application/octet-stream
Content-Length 6771845609
Connection close
Last-Modified Fri, 19 May 2023 08:55:10 GMT
ETag "6467396e-193a229e9"
X-Frame-Options DENY
X-Content-Type-Options nosniff
Accept-Ranges bytes
Strict-Transport-Security max-age=15724800; includeSubDomains
Access-Control-Allow-Origin *
Access-Control-Allow-Credentials true
Access-Control-Allow-Methods GET, PUT, POST, DELETE, PATCH, OPTIONS
Access-Control-Allow-Headers DNT,Keep-Alive,User-Agent,X-Requested-With,If-Modified-Since,Cache-Control,Content-Type,Range,Authorization
Access-Control-Max-Age 1728000


In [69]:
adm3_bounds = get_aoi(iraq_adm3_filt, world=False)
adm3_bounds

{'lon': [38.794836202000056, 48.62379496600005],
 'lat': [29.06952693900007, 37.377264006000075]}

In [71]:
# Slice the data
start_date = "2001-01-01"
end_date = "2001-05-31"

# Subset
spei_time_loc = spei_xr.sel(
    time=slice(start_date, end_date),
    lon=slice(adm3_bounds["lon"][0], adm3_bounds["lon"][1]),
    # needed to swap lat 1 and lat 0 - why?
    #lat=slice(adm3_bounds["lat"][1], adm3_bounds["lat"][0])
    )
spei_time_loc


<xarray.DataArray 'spei' (time: 5, lat: 166, lon: 197)>
[163510 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2001-01-01 2001-02-01 ... 2001-05-01
  * lon      (lon) float64 38.8 38.85 38.9 38.95 39.0 ... 48.45 48.5 48.55 48.6
  * lat      (lat) float64 37.35 37.3 37.25 37.2 37.15 ... 29.25 29.2 29.15 29.1
Attributes:
    long_name:  Standardized Precipitation-Evapotranspiration Index
    units:      z-values

In [ ]:
spei_time_loc.plot(col="time",
                             col_wrap=2,
                             figsize=(8,15),
                   cmap='RdBu')
plt.show()

In [31]:
# region is the index of the geopandas object
fin_spei = spei_time_loc.where(adm3_mask)
print(fin_spei.dims)

('time', 'lat', 'lon', 'region')


In [ ]:
mean_sum = fin_spei.groupby("time").mean(["lat", "lon"])
mean_df = mean_sum.to_dataframe()
mean_df.rename(columns = {'spei':'mean_spei'},inplace=True)


In [38]:
max_sum = fin_spei.groupby("time").max(["lat", "lon"])
max_df = max_sum.to_dataframe()
max_df.rename(columns = {'spei':'max_spei'},inplace=True)

min_sum = fin_spei.groupby("time").min(["lat", "lon"])
min_df = min_sum.to_dataframe()
min_df.rename(columns = {'spei':'min_spei'},inplace=True)

In [33]:
perc_drought = spei_time_loc.\
              where(adm3_mask & (spei_time_loc < -1)).\
              groupby("time").count(["lat", "lon"])
perc_drought_df = perc_drought.to_dataframe()
perc_drought_df.rename(columns = {'spei':'num_drought'},inplace=True)


In [ ]:
count_all = fin_spei.\
                groupby("time").count(["lat", "lon"])
count_all_df = count_all.to_dataframe()
count_all_df.rename(columns = {'spei':'num_all'},inplace=True)

In [ ]:
summaries = pd.concat([mean_df, max_df, min_df,perc_drought_df,count_all_df], axis=1)
summaries = summaries.reset_index()
summaries = summaries.merge(iraq_adm3_filt.loc[:,('ADM3_EN','ADM3_PCODE')],how='outer',
                            left_on='region',right_on=iraq_adm3_filt.index)

In [50]:
# confirm all regions assigned
print(summaries[summaries['ADM3_PCODE'].isna()].shape)
# confirm no counts of spei values in an area are more than
# the counts of spei values that meet the drought threshold
summaries[summaries['num_drought'] > summaries['num_all']]

(0, 10)


,index,time,region,mean_spei,max_spei,min_spei,num_drought,num_all,ADM3_EN,ADM3_PCODE


In [ ]:
summaries = summaries.drop(columns=['index'])
summaries.to_csv('mswep_hpet_01_spei_sums.csv',index=False)